# GRU4REC-F

This notebook trains models from `models.py` on the input data, and evaluates their performance. 

It's set to train GRU4REC-F (our proposed non-attentive model) by default. 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 23 08:39:11 2021

@author: lpott
"""
import argparse
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

from preprocessing import *
from dataset import *
from metrics import *
from model import *
from utils import bert2dict


In [3]:
# variables

read_filename ="data/movielens-1m/ratings.dat"
read_bert_filename = "data/bert_sequence_1m.txt"
read_movie_filename = ""#"movies-1m.csv"
size = "1m"

num_epochs = 50
lr =  0.005#0.01
lr_alternate = 0.001#0.001
batch_size = 64
reg = 3e-5 # was 1e-5 before
train_method = "alternate"
loss_type = "BPR_MAX"
num_neg_samples = 25
reg_bpr = 0


hidden_dim = 512
embedding_dim = 512
bert_dim= 768
window = 0

freeze_plot = False
tied = False
dropout= 0

k = 10
max_length = 200
min_len = 5


# nextitnet options...
hidden_layers = 3
dilations = [1,2,4,16]

model_type = "feature_add"

device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

In [4]:
torch.cuda.empty_cache()

In [5]:
# ------------------Data Initialization----------------------#

# convert .dat file to time-sorted pandas dataframe
ml_1m = create_df(read_filename,size=size)

# remove users who have sessions lengths less than min_len
ml_1m = filter_df(ml_1m,item_min=min_len)

========== Creating DataFrame ==========
user_id        6040
item_id        3706
rating            5
timestamp    458455
dtype: int64
(1000209, 4)
Minimum Session Length: 20
Maximum Session Length: 2314
Average Session Length: 165.60
========== Filtering Sessions <= 5  DataFrame ==========
user_id        6040
item_id        3416
rating            5
timestamp    458254
dtype: int64
(999611, 4)
Minimum Session Length: 18
Maximum Session Length: 2277
Average Session Length: 165.50
Average Item User Appearances 292.63


In [6]:
# ------------------Data Initialization----------------------#
if read_movie_filename != "":
    ml_movie_df = create_movie_df(read_movie_filename,size=size)
    ml_movie_df = convert_genres(ml_movie_df)
    
    # initialize reset object
    reset_object = reset_df()
    
    # map all user ids, item ids, and genres to range 0 - number of users/items/genres
    ml_1m,ml_movie_df = reset_object.fit_transform(ml_1m,ml_movie_df)
    
    # value that padded genre tokens shall take
    pad_genre_token = reset_object.genre_enc.transform(["NULL"]).item()
    
    genre_dim = len(np.unique(np.concatenate(ml_movie_df.genre))) - 1

else:
    # initialize reset object
    reset_object = reset_df()
    
    # map all user ids and item ids to range 0 - Number of Users/Items 
    # i.e. [1,7,5] -> [0,2,1]
    ml_1m = reset_object.fit_transform(ml_1m)
    
    pad_genre_token = None
    ml_movie_df = None
    genre_dim = 0

========== Initialize Reset DataFrame Object ==========
========== Resetting user ids and item ids in DataFrame ==========


In [7]:
# ------------------Data Initialization----------------------#
# how many unique users, items, ratings and timestamps are there
n_users,n_items,n_ratings,n_timestamp = ml_1m.nunique()

# value that padded tokens shall take
pad_token = n_items

# the output dimension for softmax layer
output_dim = n_items


# get the item id : bert plot embedding dictionary
if bert_dim != 0:
    feature_embed = bert2dict(bert_filename=read_bert_filename)

========== Reading .txt file with all item id and embeddings ==========


In [8]:
# create a dictionary of every user's session (history)
# i.e. {user: [user clicks]}
if size == "1m":
    user_history = create_user_history(ml_1m)

elif size == "20m":
    import pickle
    with open('userhistory.pickle', 'rb') as handle:
        user_history = pickle.load(handle)
# create a dictionary of all items a user has not clicked
# i.e. {user: [items not clicked by user]}
# user_noclicks = create_user_noclick(user_history,ml_1m,n_items)
# with open('filename.pickle', 'wb') as handle:
#     pickle.dump(user_history, 'userhistory.pickle', protocol=pickle.HIGHEST_PROTOCOL)

  2%|▏         | 92/6040 [00:00<00:06, 912.40it/s]

========== Creating User Histories ==========


100%|██████████| 6040/6040 [00:06<00:00, 936.88it/s]


In [9]:
# split data by leave-one-out strategy
# have train dictionary {user: [last 41 items prior to last 2 items in user session]}
# have val dictionary {user: [last 41 items prior to last item in user session]}
# have test dictionary {user: [last 41 items]}
# i.e. if max_length = 4, [1,2,3,4,5,6] -> [1,2,3,4] , [2,3,4,5] , [3,4,5,6]
train_history,val_history,test_history = train_val_test_split(user_history,max_length=max_length)

# initialize the train,validation, and test pytorch dataset objects
# eval pads all items except last token to predict
train_dataset = GRUDataset(train_history,genre_df=ml_movie_df,mode='train',max_length=max_length,pad_token=pad_token,pad_genre_token=pad_genre_token)
val_dataset = GRUDataset(val_history,genre_df=ml_movie_df,mode='eval',max_length=max_length,pad_token=pad_token,pad_genre_token=pad_genre_token)
test_dataset = GRUDataset(test_history,genre_df=ml_movie_df,mode='eval',max_length=max_length,pad_token=pad_token,pad_genre_token=pad_genre_token)

# create the train,validation, and test pytorch dataloader objects
train_dl = DataLoader(train_dataset,batch_size = batch_size,shuffle=True)
val_dl = DataLoader(val_dataset,batch_size=64)
test_dl = DataLoader(test_dataset,batch_size=64)

100%|██████████| 6040/6040 [00:00<00:00, 133237.24it/s]

========== Splitting User Histories into Train, Validation, and Test Splits ==========


In [10]:
print("Bert dim: {:d}".format(bert_dim))
print("Genre dim: {:d}".format(genre_dim))
print("Pad Token: {}".format(pad_token))
print("Pad Genre Token: {}".format(pad_genre_token))

Bert dim: 768
Genre dim: 0
Pad Token: 3416
Pad Genre Token: None


In [11]:
# ------------------Model Initialization----------------------#

# initialize gru4rec model with arguments specified earlier
if model_type == "feature_add":
    model = gru4recF(embedding_dim=embedding_dim,
             hidden_dim=hidden_dim,
             output_dim=output_dim,
             genre_dim=genre_dim,
             batch_first=True,
             max_length=max_length,
             pad_token=pad_token,
             pad_genre_token=pad_genre_token,
             bert_dim=bert_dim,
             tied = tied,
             dropout=dropout)


if model_type == "feature_concat":
    model = gru4recFC(embedding_dim=embedding_dim,
             hidden_dim=hidden_dim,
             output_dim=output_dim,
             genre_dim=genre_dim,
             batch_first=True,
             max_length=max_length,
             pad_token=pad_token,
             pad_genre_token=pad_genre_token,
             bert_dim=bert_dim,
             tied = tied,
             dropout=dropout)

if model_type == "vanilla":
    model = gru4rec_vanilla(hidden_dim=hidden_dim,
                            output_dim=output_dim,
                            batch_first=True,
                            max_length=max_length,
                            pad_token=pad_token,
                            tied=tied,
                            embedding_dim=embedding_dim,
                           device=device)

if model_type =="feature_only":
    model = gru4rec_feature(hidden_dim=hidden_dim,
                            output_dim=output_dim,
                            batch_first=True,
                            max_length=max_length,
                            pad_token=pad_token,
                            bert_dim=bert_dim)

if model_type == "conv":
    model = gru4rec_conv(embedding_dim,
                 hidden_dim,
                 output_dim,
                 batch_first=True,
                 max_length=200,
                 pad_token=0,
                 dropout=0,
                 window=3,
                 tied=tied)
    
if model_type == "nextitnet":
    model = NextItNet(embedding_dim=embedding_dim,
                      output_dim=output_dim,
                      hidden_layers=hidden_layers,
                      dilations=dilations,
                      pad_token=n_items,
                      max_len=max_length)

In [12]:
if bert_dim != 0:
    model.init_weight(reset_object,feature_embed)
    
model = model.to(device)

In [13]:
# initialize Adam optimizer with gru4rec model parameters
if train_method != "normal":
    optimizer_features = torch.optim.Adam([param for name,param in model.named_parameters() if (("movie" not in name) or ("plot_embedding" in name) or ("genre" in name)) ],
                                          lr=lr_alternate,weight_decay=reg)
    
    optimizer_ids = torch.optim.Adam([param for name,param in model.named_parameters() if ("plot" not in name) and ("genre" not in name)],
                                     lr=lr,weight_decay=reg)

elif train_method == "normal":
    optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=reg)
    
if freeze_plot and bert_dim !=0:
    model.plot_embedding.weight.requires_grad = False

In [14]:
if loss_type == "XE":
    loss_fn = nn.CrossEntropyLoss(ignore_index=n_items)
    
elif loss_type == "BPR":
    loss_fn = BPRLoss(user_history = user_history,
                      n_items = n_items, 
                      df = ml_1m,
                      device = device, 
                      samples=num_neg_samples)

elif loss_type == "BPR_MAX":
    loss_fn = BPRMaxLoss(user_history = user_history,
                      n_items = n_items, 
                      df = ml_1m,
                      device = device,
                      reg = reg_bpr,
                      samples=num_neg_samples)

3416


In [15]:
Recall_Object = Recall_E_prob(ml_1m,user_history,n_users,n_items,k=k,device=device)

========== Creating Hit@10 Metric Object ==========


In [16]:
#print("Baseline POP results: ",Recall_Object.popular_baseline())

In [17]:
#training_hit = Recall_Object(model,train_dl)
#validation_hit = Recall_Object(model,val_dl)
#testing_hit = Recall_Object(model,test_dl)
#print("Training Hits@{:d}: {:.2f}".format(k,training_hit))
#print("Validation Hits@{:d}: {:.2f}".format(k,validation_hit))
#print("Testing Hits@{:d}: {:.2f}".format(k,testing_hit))

In [18]:
# ------------------Training Initialization----------------------#
max_train_hit = (0,0,0)
max_val_hit = (0,0,0)
max_test_hit = (0,0,0)

max_train_ndcg = (0,0,0)
max_val_ndcg = (0,0,0)
max_test_ndcg = (0,0,0)

max_train_mrr = 0
max_val_mrr = 0
max_test_mrr = 0
i = 0;
for epoch in range(num_epochs):
    print("="*20,"Epoch {}".format(epoch+1),"="*20)
    
    model.train()  
    
    running_loss = 0

    for j,data in enumerate(tqdm(train_dl,position=0,leave=True)):
        
        if train_method != "normal":
            optimizer_features.zero_grad()
            optimizer_ids.zero_grad()
            
        elif train_method == "normal": 
            optimizer.zero_grad()
        
        if genre_dim != 0:            
            inputs,genre_inputs,labels,x_lens,uid = data
            outputs = model(x=inputs.to(device),x_lens=x_lens.squeeze().tolist(),x_genre=genre_inputs.to(device))
        
        elif genre_dim == 0:
            inputs,labels,x_lens,uid = data 
            outputs = model(x=inputs.to(device),x_lens=x_lens.squeeze().tolist())
       
        if tied:
            outputs_ignore_pad = outputs[:,:,:-1]
            if loss_type == "XE":
                loss = loss_fn(outputs_ignore_pad.view(-1,outputs_ignore_pad.size(-1)),labels.view(-1).to(device))
            elif loss_type == "BPR" or loss_type == "BPR_MAX":
                loss = loss_fn(outputs,labels.to(device),x_lens,uid)

            
        else:
            if loss_type == "XE":
                loss = loss_fn(outputs.view(-1,outputs.size(-1)),labels.view(-1).to(device))
            elif loss_type == "BPR" or loss_type == "BPR_MAX":   
                loss = loss_fn(outputs,labels.to(device),x_lens,uid)

        loss.backward()
        
        
        if train_method != "normal":
            if train_method == "interleave":
                # interleave on the epochs
                if (j+1) % 2 == 0:
                    optimizer_features.step()
                else:
                    optimizer_ids.step()

            elif train_method == "alternate":
                if (epoch+1) % 2 == 0:
                    optimizer_features.step()
                else:
                    optimizer_ids.step()
        
    
                    
        elif train_method == "normal":
            optimizer.step()

        running_loss += loss.detach().cpu().item()

    del outputs
    torch.cuda.empty_cache()
    training_hit,training_ndcg,training_mrr = Recall_Object(model,train_dl,"train")
    validation_hit,validation_ndcg,validation_mrr = Recall_Object(model,val_dl,"validation")
    testing_hit,testing_ndcg,testing_mrr = Recall_Object(model,test_dl,"test")
    
    if max_val_mrr < validation_mrr:
        max_val_hit = validation_hit
        max_test_hit = testing_hit
        max_train_hit = training_hit
        
        max_train_ndcg = training_ndcg
        max_val_ndcg = validation_ndcg
        max_test_ndcg = testing_ndcg
        
        max_train_mrr = training_mrr
        max_val_mrr = validation_mrr
        max_test_mrr = testing_mrr
        
        torch.save(model.state_dict(),"best_model_{}.pt".format(size))
        print("BEST MODEL PERFORMANCE")
    
    torch.cuda.empty_cache()
    print("Training Loss: {:.5f}".format(running_loss/len(train_dl)))
    
    print("Train Hits \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*training_hit))
    print("Train ndcg \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*training_ndcg))
    print("Train mrr \t {:.5f}".format(training_mrr))


    print("Valid Hits \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*validation_hit))
    print("Valid ndcg \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*validation_ndcg))
    print("Valid mrr \t {:.5f}".format(validation_mrr))

    print("Test Hits \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*testing_hit))
    print("Test ndcg \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*testing_ndcg))
    print("Test mrr \t {:.5f}".format(testing_mrr))
    
print("="*100)
print("Maximum Training Hit \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*max_train_hit))
print("Maximum Validation Hit \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*max_val_hit))
print("Maximum Testing Hit \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*max_test_hit))

  0%|          | 0/95 [00:00<?, ?it/s]

==================== Epoch 1 ====================


  0%|          | 0/95 [00:00<?, ?it/s]

BEST MODEL PERFORMANCE
Training Loss: 0.47140
Train Hits 	 @10: 0.76887 	 @5 : 0.64768 	 @1 : 0.31937
Train ndcg 	 @10: 0.53287 	 @5 : 0.49356 	 @1 : 0.31937
Train mrr 	 0.46924
Valid Hits 	 @10: 0.67533 	 @5 : 0.54884 	 @1 : 0.24503
Valid ndcg 	 @10: 0.44542 	 @5 : 0.40432 	 @1 : 0.24503
Valid mrr 	 0.38670
Test Hits 	 @10: 0.62765 	 @5 : 0.50844 	 @1 : 0.23460
Test ndcg 	 @10: 0.41724 	 @5 : 0.37868 	 @1 : 0.23460
Test mrr 	 0.36651
==================== Epoch 2 ====================


  0%|          | 0/95 [00:00<?, ?it/s]

...

Training Loss: 0.34373
Train Hits 	 @10: 0.82384 	 @5 : 0.70629 	 @1 : 0.34619
Train ndcg 	 @10: 0.57568 	 @5 : 0.53749 	 @1 : 0.34619
Train mrr 	 0.50576
Valid Hits 	 @10: 0.69652 	 @5 : 0.56490 	 @1 : 0.24818
Valid ndcg 	 @10: 0.45831 	 @5 : 0.41552 	 @1 : 0.24818
Valid mrr 	 0.39644
Test Hits 	 @10: 0.66209 	 @5 : 0.53907 	 @1 : 0.24752
Test ndcg 	 @10: 0.43883 	 @5 : 0.39891 	 @1 : 0.24752
Test mrr 	 0.38289
==================== Epoch 8 ====================


  0%|          | 0/95 [00:00<?, ?it/s]

BEST MODEL PERFORMANCE
Training Loss: 0.31211
Train Hits 	 @10: 0.84901 	 @5 : 0.74371 	 @1 : 0.38576
Train ndcg 	 @10: 0.61204 	 @5 : 0.57768 	 @1 : 0.38576
Train mrr 	 0.54417
Valid Hits 	 @10: 0.71060 	 @5 : 0.58576 	 @1 : 0.27368
Valid ndcg 	 @10: 0.47937 	 @5 : 0.43893 	 @1 : 0.27368
Valid mrr 	 0.41945
Test Hits 	 @10: 0.68030 	 @5 : 0.56358 	 @1 : 0.26606
Test ndcg 	 @10: 0.45982 	 @5 : 0.42206 	 @1 : 0.26606
Test mrr 	 0.40371
==================== Epoch 9 ====================


100%|██████████| 95/95 [00:59<00:00,  1.60it/s]


KeyboardInterrupt: 

In [ ]:
print("="*100)
print("Maximum Train Hit \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*max_train_hit))
print("Maximum Valid Hit \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*max_val_hit))
print("Maximum Test Hit \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*max_test_hit))

print("Maximum Train NDCG \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*max_train_ndcg))
print("Maximum Valid NDCG \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*max_val_ndcg))
print("Maximum Test NDCG \t @10: {:.5f} \t @5 : {:.5f} \t @1 : {:.5f}".format(*max_test_ndcg))

print("Maximum Train MRR \t {:.5f}".format(max_train_mrr))
print("Maximum Valid MRR \t {:.5f}".format(max_val_mrr))
print("Maximum Test MRR \t {:.5f}".format(max_test_mrr))

## Create some visual examples ... pray to god they are correct!

In [ ]:
import os
os.getcwd()

In [ ]:
movie_df =pd.read_csv("data/movielens-1m/movies-1m.csv")

In [ ]:
model.load_state_dict(torch.load("best_model_1m.pt"))

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
with torch.no_grad():
    mode = 'validation'
    model.eval()
    for data in val_dl:
        if model.genre_dim != 0:            
            inputs,genre_inputs,labels,x_lens,uid = data
            outputs = model(x=inputs.to(Recall_Object.device),x_lens=x_lens.squeeze().tolist(),x_genre=genre_inputs.to(Recall_Object.device))

        else:
            inputs,labels,x_lens,uid = data
            outputs = model(x=inputs.to(Recall_Object.device),x_lens=x_lens.squeeze().tolist())
            
        for i,uid in enumerate(uid.squeeze()):
            history = Recall_Object.user_history[uid.item()]

            if mode == "train":
                history = set(history[:-2])

            if mode == "validation":
                history = set(history[:-1])

            if mode == "test":
                history = set(history)


            #sample_negatives = []
            sample_negatives = [labels[i,x_lens[i].item()-1].item()]
            while len(sample_negatives) < 101:

                sampled_ids = np.random.choice(Recall_Object.n_items, 100, replace=False, p=Recall_Object.p).tolist()
                sampled_ids = [x for x in sampled_ids if x not in history and x not in sample_negatives]
                sample_negatives.extend(sampled_ids[:])

            #sample_negatives = sample_negatives[:100].copy()
            sample_negatives = sample_negatives[:101].copy()
            #sample_negatives.append(labels[i,x_lens[i].item()-1].item())
                    

                    


                        
            rank = torch.where(outputs[i,x_lens[i].item()-1,sample_negatives].argsort(descending=True)==0)[0].item()  

            
            movies_ranked = np.array(sample_negatives)[outputs[i,x_lens[i].item()-1,sample_negatives].argsort(descending=True).cpu().numpy()]
            gt_movie = labels[i,x_lens[i].item()-1].item()
            user_id = uid.item()
            user_input_session = inputs[i][inputs[i] != 3706].cpu().numpy()
            
            print("User ID: ",user_id)
            print("Rankings at t+1: ",movies_ranked)
            print("GT Item: ",gt_movie)
            print("GT Item Rank: ",rank)
            print("User Session: ",user_input_session)
            print("Last 20 movies: \n",movie_df.loc[user_input_session,['title','genres']][-20:])    #movie_df.title[user_input_session][-20:].values)
            print("Top 10 recommended:\n",movie_df.loc[movies_ranked,['title','genres']][:10])

            print("="*100)
            
            print

In [ ]:
outputs.shape

In [ ]:
movie_df.title[movies_ranked][:10].values,movie_df.genres[movies_ranked][:10].values

In [ ]:
movie_df.loc[movies_ranked,['title','genres']][:10] #  .title[movies_ranked][:10].values,movie_df.genres[movies_ranked][:10].values

In [ ]:
toy_example = [1050,0,2120,2072,711,2073,351,2285,3542,3682,3685,3327,1346,2286] 

In [ ]:
toy_example_len = torch.LongTensor([len(toy_example)])

In [ ]:
toy_example = torch.LongTensor([toy_example + [3706] * (200-toy_example_len)])

In [ ]:
outputs = model(x=toy_example.to(Recall_Object.device),x_lens=toy_example_len.tolist())

In [ ]:
movie_df[movie_df.genres == "Animation|Children's|Comedy"]

In [ ]:
movie_df.iloc[ [1050,0,2120,2072,711,2073,351,2285,3542,3682,3685,3327,1346,2286] ,:]

In [ ]:
torch.where(outputs.squeeze()[toy_example_len].squeeze().argsort(descending=True) == 2286)

In [ ]:
movie_df.iloc[3582]

In [ ]:
movie_df.iloc[outputs.squeeze()[toy_example_len].squeeze().argsort(descending=True).cpu().numpy()][:10]